In [ ]:
import pandas as pd
import numpy as np 
from bs4 import BeautifulSoup
import requests
from datetime import datetime 
import time
import pytz
import os

In [43]:
cat_history_df = pd.read_csv('cat_history.csv')

In [44]:
cat_history_df.head()

,id,url,name,age,gender,cafe_cat,kid_approved,dog_approved,companion_cat,bonded_pair,single_cat,type,description,system_import_at
0,18576198,https://catcafebk.com/our-cats/?cat=18576198,Aeon,10 Years 5 Months,Female,False,False,False,True,False,True,Domestic Short Hair,"Aeon is a gorgeous, chatty tortie girl with a ...",2023-01-22 19:13:48+00:00
1,18949108,https://catcafebk.com/our-cats/?cat=18949108,Aioli & Dijon,4 Months 3 Days,NaN,False,True,True,False,True,False,Domestic Short Hair,Aioli & Dijon are full of zest and zing — whet...,2023-01-22 19:13:48+00:00
2,18049093,https://catcafebk.com/our-cats/?cat=18049093,Apollo & Artemis,1 Year 1 Month,NaN,False,True,True,False,True,False,Domestic Short Hair / Mixed,Apollo and Artemis are sweet and sassy teens r...,2023-01-22 19:13:48+00:00
3,18844592,https://catcafebk.com/our-cats/?cat=18844592,Baby Shark,9 Months 7 Days,Female,False,False,False,False,False,False,Domestic Short Hair,Baby Shark is a sweet tuxedo who likes taking ...,2023-01-22 19:13:48+00:00
4,18949086,https://catcafebk.com/our-cats/?cat=18949086,Bada & Pow,4 Months 19 Days,Male,False,True,True,False,True,False,Domestic Short Hair,"Bada and Pow are two adorable, playful brother...",2023-01-22 19:13:48+00:00


In [46]:
cat_history_df.loc[110:116]

,id,url,name,age,gender,cafe_cat,kid_approved,dog_approved,companion_cat,bonded_pair,single_cat,type,description,system_import_at
110,18435919,https://catcafebk.com/our-cats/?cat=18435919,Ube & Steal,9 Months 9 Days,NaN,False,True,True,False,True,False,Domestic Short Hair,Ube and Steal can't wait to cuddle with you! S...,2023-01-22 19:13:48+00:00
111,18667410,https://catcafebk.com/our-cats/?cat=18667410,Waldorf,13 Years 10 Months,Male,False,True,True,True,False,True,Domestic Short Hair,Cat? Rabbit? Muppet? Waldorf is a senior cat w...,2023-01-22 19:13:48+00:00
112,18568223,https://catcafebk.com/our-cats/?cat=18568223,Where & Wal,8 Months 24 Days,NaN,False,True,False,False,True,False,Domestic Short Hair,"Where and Wal aren’t siblings, but they were c...",2023-01-22 19:13:48+00:00
113,18634866,https://catcafebk.com/our-cats/?cat=18634866,Will & Way,8 Months 6 Days,NaN,False,True,True,False,True,False,Domestic Short Hair,Will and Way are paws down the most affectiona...,2023-01-22 19:13:48+00:00
114,18895748,https://catcafebk.com/our-cats/?cat=18895748,Winston & Venkman,4 Months 21 Days,NaN,False,True,True,False,True,False,Domestic Short Hair,Winston is the charmer of the duo. She’s more ...,2023-01-22 19:13:48+00:00
115,18587232,https://catcafebk.com/our-cats/?cat=18587232,Wonky & Wheezy,9 Months 17 Days,NaN,False,True,True,False,True,False,Domestic Short Hair,Wonky may be the cuddliest kitten in the world...,2023-01-22 19:13:48+00:00
116,18860609,https://catcafebk.com/our-cats/?cat=18860609,Zooey Deschatnel,2 Years 9 Months,Female,False,False,False,True,False,True,Domestic Short Hair,Zooey was a past colony cat who was found dirt...,2023-01-22 19:13:48+00:00


In [49]:
cat_history_df[cat_history_df['id'] == 18469036].loc[109]['description']

'TW and Sapling will certainly add some cuteness and excitement to your life. They love playing and wrestling with each other and get along extremely well with other cats in the household. Sapling likes to be close to people (she’s almost always in the same room as her foster), but isn’t much of a cuddler. TW is more independent, but will curl right up next to you on the couch.\xa0\n\nBoth cats are gorgeous, sleek black cats. Sapling has red undertones to his slightly fluffier coat and TW has adorable white markings on her belly (they’ve been described as a little bikini).\xa0\n\nThese kittens love an opportunity to play around with either ribbons or “real” cat toys.'

In [29]:
cat_history_df.system_import_at.unique()

array(['2023-01-22 19:13:48+00:00', '2023-01-22 19:49:42+00:00',
       '2023-01-22 19:51:39+00:00'], dtype=object)

In [30]:
cat_history_df.shape

(352, 17)

In [32]:
last_mask = (cat_history_df['system_import_at'] == '2023-01-22 19:49:42+00:00')
this_mask = (cat_history_df['system_import_at'] == '2023-01-22 19:51:39+00:00')

In [37]:
last_cats = cat_history_df[last_mask]['name'].unique()
this_cats = cat_history_df[this_mask]['name'].unique()

diff = [x for x in this_cats if x not in last_cats]
print(diff) 

['Ernutter & Andolf']


### Testing Foster List Data Collection

In [193]:
import sys
sys.path.insert(1, '../src')
from daily_cats import *
import re

In [194]:
soup = getCatPage('https://bbawcfosterteam.wixsite.com/cats/availablecats')

In [195]:
cats = soup.find('fluid-columns-repeater').find_all('div', class_ = 'comp-kjevao4x5 YzqVVZ')

results_list = []

for cat in cats: 
    text_data = cat.find('div', class_='hFQZVn comp-kjevao571 wixui-box')
    cat_image = cat.find('img').attrs['src']
    cat_link = cat.find('a').attrs['href']
    
    # get basic info
    cat_name = text_data.find('h4').text
    cat_summary = text_data.find('div', class_="BaOVQ8 tz5f0K comp-kjevl5nv wixui-text").text
    cat_timing = text_data.find('div', class_="BaOVQ8 tz5f0K comp-kjevrpz5 wixui-text").text
    cat_description = text_data.find('div', class_="BaOVQ8 tz5f0K comp-kjevao593 wixui-text").text
    results_list.append(
        {'cat_name':cat_name,
         'cat_summary':cat_summary,
         'cat_timing':cat_timing,
         'cat_description':cat_description,
         'cat_image':cat_image,
         'cat_link':cat_link}
    )
    
    # go to page for additional info
    # cat_page_url = cat.find('a').attrs['href']
    # cat_soup = getCatPage(cat_page_url)
    
    # img_link = cat_soup.find('img').attrs['src']
    
    
    

# # get image 
# obj.find('a').find('img')


In [196]:
test = pd.json_normalize(results_list)
test

,cat_name,cat_summary,cat_timing,cat_description,cat_image,cat_link
0,Amewlia Earhart,2 year-old tabby lady,Ready evening of 3/8 at the cafe,This sweet mom is so ready for retirement! Liv...,https://static.wixstatic.com/media/7f467d_2ec4...,https://bbawcfosterteam.wixsite.com/cats/avali...
1,Bobbin,11 month-old dilute calico lady,"Ready now, at the cafe",This super beautiful lady would love nothing m...,https://static.wixstatic.com/media/7f467d_e8d0...,https://bbawcfosterteam.wixsite.com/cats/avali...
2,Castle,8 year-old tabby lady,Needs a new home by 2/28,From her fosterer: Castle is the sweetest frie...,https://static.wixstatic.com/media/7f467d_cd0d...,https://bbawcfosterteam.wixsite.com/cats/avali...
3,Charlie Chaplin,Adult all-tabby male,"Ready now, at the cafe",Found lost and alone and trying to eat grass o...,https://static.wixstatic.com/media/7f467d_8c2f...,https://bbawcfosterteam.wixsite.com/cats/avali...
4,Clara and Bella,8 month-old white and black sisters,Need a home by 4/2,From their fosterer: Bella and Clara are the s...,https://static.wixstatic.com/media/7f467d_7957...,https://bbawcfosterteam.wixsite.com/cats/avali...
5,DJ,4 year-old white and orange male,"Ready now, at the cafe",Help this guy out with a new lease on life! Fr...,https://static.wixstatic.com/media/7f467d_22a2...,https://bbawcfosterteam.wixsite.com/cats/avali...
6,Dashiell Hammett and Raymond Chandler,4 month-old BFFs,"Ready now, at the cafe","I mean, what can I say. They are too cute. Thi...",https://static.wixstatic.com/media/7f467d_f372...,https://bbawcfosterteam.wixsite.com/cats/avali...
7,Earthquake and Wildfire,5 month-old siblings,"Ready now, fosterer can meet you at the cafe",Say hello to Earthquake and Firestorm! This bo...,https://static.wixstatic.com/media/7f467d_2767...,https://bbawcfosterteam.wixsite.com/cats/avali...
8,Jemma (hold),4-ish year old tuxedo lady,"Ready now, we'll coordinate her rescue once yo...",Jemma has been living in a Home Depot parking ...,https://static.wixstatic.com/media/7f467d_0ae8...,https://bbawcfosterteam.wixsite.com/cats/avali...
9,Maravilla,1 year-old long haired tabby lady,"Ready now, at the cafe",Check out this sassy floof! Maravilla has the ...,https://static.wixstatic.com/media/7f467d_1eb9...,https://bbawcfosterteam.wixsite.com/cats/avali...


In [192]:
test.tail(1)['cat_link'].values

array(['https://bbawcfosterteam.wixsite.com/cats/avaliblecats/tully'],
      dtype=object)

In [165]:
cafe_prospect = cat_soup.find('div', class_='comp-kjevammt2 YzqVVZ')


health_concerns = cat_soup.find('div', class_='comp-kjevamml YzqVVZ')


upcoming_appointments = cat_soup.find('div', class_='comp-kjevammp3 YzqVVZ')



In [170]:
upcoming_appointments.find('h4').text

'Vaccine'

In [ ]:
cats[1].find('a').

In [156]:
cats[1].find('a').attrs['href']

'https://bbawcfosterteam.wixsite.com/cats/avaliblecats/castle'